<a href="https://colab.research.google.com/github/edgi-govdata-archiving/PipelineSafety/blob/phmsa_location_pdf/phmsa_location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install pypdf
!pip install geotext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.6 MB/s eta 0:00:00


How many and which enforcement actions are corrective action orders?

In [20]:
import pandas as pd

# load data
phmsa = pd.read_csv("/content/PHMSA_Pipeline_Enforcement_Raw_Data-CC - PHMSA_Pipeline_Enforcement_Raw_Data-2.csv")

#define criteria for Corrective_Action_Order_Ind = YES
CAO = phmsa[phmsa["Corrective_Action_Order_Ind"] == "Yes"]

# extract the text before first hyphen
CAO["Material"] = CAO["Report_Number"].str.split("-").str[0].str.strip()

# count frequencies
material_counts = CAO["Material"].value_counts()

print(material_counts)

Material
Hazardous Liquid                  85
Gas Transmission and Gathering    54
LNG                                1
Name: count, dtype: int64


/tmp/ipython-input-3664189451.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CAO["Material"] = CAO["Report_Number"].str.split("-").str[0].str.strip()


Quick math check

In [22]:
CAO.shape

(152, 54)

In [21]:
85+54+1

140

How many and which enforcement actions have incident reports?

In [23]:
incident_reports = phmsa[phmsa["Report_Type"] == "Incident Report"]

incident_reports

,CPF_Number,Operator_ID,Operator_Name,Operator_Searchable_Name,Region,Pipeline_Type,How_Discovered?,Case_Type,Notice_Actions,Cited_Regulations,...,Case_Closure_Ind,Proposed_Civil_Penalty_Ind,Proposed_Compliance_Order_Ind,Penalty_Reduction_Ind,Basis_for_penalty_reduction,Report_Type,Report_Number,WMS_Activity_Seq,Data_Source,Data_as_of
51,12025018WL,1845,"BUCKEYE PARTNERS, LP","BUCKEYE PARTNERS, LP",Eastern,INTERSTATE LIQUID,Failure Investigation,Warning Letter,WI,195.55(a)(6),...,Yes,NaN,NaN,NaN,NaN,Incident Report,Hazardous Liquid - 20230062,23-268982,WMS,09/03/2025 9:00 AM
69,32025018CAO,32334,SOUTH BOW INFRASTRUCTURE OPERATIONS INC.,SOUTH BOW INFRASTRUCTURE OPERATIONS INC.,Central,INTERSTATE LIQUID,Failure Investigation,Corrective Action Order,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Incident Report,Hazardous Liquid - 20250112,25-338014,WMS,09/03/2025 9:00 AM
70,42025054NOPSO,18718,SUNOCO PIPELINE L.P.,SUNOCO PIPELINE L.P.,Southwest,INTERSTATE LIQUID,Failure Investigation,Safety Order,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Incident Report,Hazardous Liquid - 20250051,25-332048,WMS,09/03/2025 9:00 AM
93,42024033NOPV,32206,"FREEPORT LNG DEVELOPMENT, L.P.","FREEPORT LNG DEVELOPMENT, L.P.",Southwest,INTERSTATE LNG,Failure Investigation,Notice of Probable Violation,CP,"193.2507, 193.2513(a), 193.2619(e), 193.271...",...,Yes,Yes,NaN,NaN,NaN,Incident Report,LNG - 20220004,22-245663,WMS,09/03/2025 9:00 AM
109,32024008NOPV,10012,NUSTAR PIPELINE OPERATING PARTNERSHIP L.P.,NUSTAR PIPELINE OPERATING PARTNERSHIP L.P.,Central,INTERSTATE LIQUID,Failure Investigation,Notice of Probable Violation,"CO/CP, CP","195.452(h)(4)(iv), 195.52(a)(3), 195.52(c)",...,NaN,Yes,Yes,NaN,NaN,Incident Report,Hazardous Liquid - 20220065,22-237074,WMS,09/03/2025 9:00 AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4874,320025007H,12127,MARATHON ASHLAND PIPE LINE LLC,MARATHON ASHLAND PIPE LINE LLC,Central,INTERSTATE LIQUID ONSHORE,Failure Investigation,Corrective Action Order,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Incident Report,Hazardous Liquid - 20020151,02-003624S,WMS (SMART migrated),09/03/2025 9:00 AM
4906,320021003H,6660,GREAT LAKES GAS TRANSMISSION CO,GREAT LAKES GAS TRANSMISSION LIMITED PARTNERSHIP,Central,GAS INTERSTATE ONSHORE,Failure Investigation,Corrective Action Order,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Incident Report,Gas Transmission and Gathering - 20020030,02-003704S,WMS (SMART migrated),09/03/2025 9:00 AM
4928,520020001,18536,SOUTHWEST GAS CORP,SOUTHWEST GAS CORP,Western,GAS INTRASTATE ONSHORE,Failure Investigation,Notice of Probable Violation,CP,191.5(a),...,Yes,Yes,NaN,Yes,Additional evidence,Incident Report,Gas Distribution - 20020014,04-003600S,WMS (SMART migrated),09/03/2025 9:00 AM
4938,120020001M,17360,"RICHMOND, CITY OF","RICHMOND, CITY OF",Eastern,GAS INTRASTATE ONSHORE,Failure Investigation,Notice of Amendment,NOA,192.613(b),...,Yes,NaN,NaN,NaN,NaN,Incident Report,Gas Distribution - 20010184,01-003597S,WMS (SMART migrated),09/03/2025 9:00 AM


The code below can be used to extract location information from PHMSA enforcement action PDFs. I haven't figured out how to scrape pdf data from https://primis.phmsa.dot.gov/enforcement-data/cases

The code above was written by a human, the code below was written with the help of AI.

In [31]:
import os
import re
import pandas as pd
from pypdf import PdfReader
from geotext import GeoText

US_STATES = [
    "Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware",
    "Florida","Georgia","Hawaii","Idaho","Illinois","Indiana","Iowa","Kansas","Kentucky",
    "Louisiana","Maine","Maryland","Massachusetts","Michigan","Minnesota","Mississippi",
    "Missouri","Montana","Nebraska","Nevada","New Hampshire","New Jersey","New Mexico",
    "New York","North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
    "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah","Vermont",
    "Virginia","Washington","West Virginia","Wisconsin","Wyoming"
]

def extract_location_from_pdf(pdf_path):
    # read first page text
    reader = PdfReader(pdf_path)
    try:
        text = reader.pages[0].extract_text() or ""
    except Exception:
        text = ""
    text = " ".join(text.split())  # normalize whitespace

    # find "inspected" (word boundary, case-insensitive)
    m = re.search(r"\binspected\b", text, flags=re.IGNORECASE)
    if not m:
        return None, None, None

    # start AFTER the word "inspected"
    start = m.end()

    # take up to the first period after "inspected"
    period_idx = text.find(".", start)
    if period_idx == -1:
        snippet = text[start:].strip()
    else:
        snippet = text[start:period_idx].strip()

    # remove leading punctuation/comma/colon and common prepositions like "at", "in"
    snippet = re.sub(r'^[\s,;:-]+', '', snippet)
    snippet = re.sub(r'^(?:at|in|on|near|inside|outside)\b[\s,:-]*', '', snippet, flags=re.IGNORECASE)
    raw_snippet = snippet  # keep for debug / inspection

    # get city via GeoText
    places = GeoText(snippet)
    city = places.cities[0] if places.cities else None

    # match full-state names from US_STATES (case-insensitive, escaped)
    matched_states = []
    for st in US_STATES:
        pattern = rf"\b{re.escape(st)}\b"
        if re.search(pattern, snippet, flags=re.IGNORECASE):
            matched_states.append(st)

    # If multiple states matched, join them; otherwise None
    state = ", ".join(matched_states) if matched_states else None

    return city, state, raw_snippet


def process_pdfs_in_folder(folder_path):
    rows = []
    for fname in os.listdir(folder_path):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(folder_path, fname)
        city, state, raw_snippet = extract_location_from_pdf(path)
        rows.append({
            "pdf_name": fname,
            "city": city,
            "state": state,
            "raw_snippet": raw_snippet
        })
    return pd.DataFrame(rows)


# apply to folder
folder = "/content/pdf"
df = process_pdfs_in_folder(folder)
print(df)
# optional: df.to_csv("inspected_locations.csv", index=False)


          pdf_name        city         state  \
0  42025041NOA.pdf  Providence  Rhode Island   

                                         raw_snippet  
0  Niagara Mohawk Power Corporation’s1 (Niagara) ...  
